In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path={'executable_path':ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\jlubr\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
url='https://chatham.bottleking.com/main.asp?request=search&type=w'
browser.visit(url)

In [4]:
wine_df=pd.DataFrame(columns=['name','country','region','state','subtype',
                              'varietal','price','description','merchant','address'])
merchant='Chatham BottleKing'
address='41 Watchung Ave, Chatham, NJ, 07928'

print("Scraping please be patient")
while True:
    html=browser.html
    soup=BeautifulSoup(html,'html.parser')
    wine_summary_boxes=soup.find_all('td',{'colspan':['2']})
    wine_summary_boxes.pop()
    for box in wine_summary_boxes:
        name=box.find('a',{'class':['Srch-producttitle']}).text
        price=box.find('span',{'class':['RegularPrice']}).text
        description=box.find('span',{'class':['Srch-partialDesc']}).text
        wine=box.find('a',{'class':['Srch-producttitle']})
        url=wine.get('href')
        try:
            browser.visit(url)
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            wineinfo=soup.find('div',{'class':['info_list']})
            details=wineinfo.find_all('span',{'class':['iteminfocat']})
            bottle_info=wineinfo.find_all('a',{'class':['reglink']})
            for detail in details:
                index=0
                subregion='N/A'
                if (detail.text =='Sub-Region:'):
                    index=1
                    subregion=bottle_info[3].text
                if (detail.text =='Country:'):
                    country=bottle_info[index].text
                if (detail.text =='Region:'):
                    region=bottle_info[index+1].text
                if (detail.text =='Grape Varietal:'):
                    varietal=bottle_info[index+2].text
                if (detail.text =='Type:'):
                    wtype=bottle_info[index+3].text
            browser.back()
        except:
            country='N/A'
            region='N/A'
            subregion='N/A'
            variety='N/A'
            wtype='N/A'
            browser.back()
            
        wine_df=wine_df.append({
            'name':name,
            'country':country,
            'region':subregion,
            'state':region,
            'subtype':wtype,
            'varietal':varietal,
            'price':price,
            'description':description,
            'merchant':merchant,
            'address':address
        },ignore_index=True)
        
    try:
        browser.links.find_by_partial_text('>>').click()
    except:
        break
browser.quit()
print('Scraping Complete')
wine_df.to_csv("chathambuyrite.csv")

Scraping please be patient
Scraping Complete
